In [2]:
# Import the MTCNN module from the facenet_pytorch package
from facenet_pytorch import MTCNN, InceptionResnetV1

# Import the torch library
import torch

# Import the datasets module from torchvision
from torchvision import datasets

# Import the DataLoader class from the torch.utils.data module
from torch.utils.data import DataLoader

# Import the Image module from PIL (Python Image Library)
from PIL import Image

# Import the OpenCV library for image processing
import cv2

# Import the time library for timing purposes
import time

# Import the os library for file and directory handling
import os

In [3]:
# initializing MTCNN and InceptionResnetV1 
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False if multiple faces keep all false and detect one only
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True to keep all faces in form of list
resnet = InceptionResnetV1(pretrained='vggface2').eval()  # using vggface2 pretrained model 

# Read data from folder

path = 'C:\\Users\\Dakhel\\'
dataset = datasets.ImageFolder(path + "FPhoto") # photos folder path 

In [15]:

idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True) 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0)) 
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx])        

# save data
data = [embedding_list, name_list] 
torch.save(data, 'data.pt') # saving data.pt file
